In [1]:
import pandas as pd
import re
import string
import numpy as np
from sklearn import  preprocessing, ensemble
import matplotlib.pyplot as plt

%matplotlib inline

In [11]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [2]:
train_df = pd.read_json("../../kaggleData/2sigma/train.json")
test_df = pd.read_json("../../kaggleData/2sigma/test.json")

In [3]:
#feature processing functions
#define punctutaion filter
def removePunctuation(x):
    #filter the head or tail blanks
    x = re.sub(r'^\s+',r' ',x)
    x = re.sub(r'\s+$',r' ',x)
    
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars, warning if you are dealing with other languages!!!!!!!!!!!!!!!
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    #change all the blank to space
    x = re.sub(r'\s',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    removing = string.punctuation#.replace('-','')# except '-'
    removed = re.sub("["+removing+"]", "", x)
    #removing the line-changing
    #removed = re.sub('\\n'," ",removed)    
    return removed

#feature processing functions
def proecessStreet(address):
    #remove the building number
    street = address.strip()
    pattern = re.compile('^[\d-]+[\D]?[\s]+')
    street = removePunctuation(pattern.sub('',street))
    #sub the st to street
    pattern = re.compile('( st)$')
    street = pattern.sub(' street',street)
    #sub the ave to avenue
    pattern = re.compile('( ave)$')
    street = pattern.sub(' avenue',street)
    #nth -> n
    #nst -> n
    #nrd -> n
    #nnd -> n
    pattern = re.compile('(\d+)((th)|(st)|(rd)|(nd))')
    street = pattern.sub('\g<1>',street)
    #deal with the w 14 street => west 14 street
    pattern = re.compile('(w)(\s+)(\d+)')    
    street = pattern.sub('west \g<3>',street)
    #deal with the e....
    pattern = re.compile('(e)(\s+)(\d+)')    
    street = pattern.sub('east \g<3>',street).strip()
    
    street = street
    
    return street
    
def getStreetNumber(address):
    #get building id in the front, return -1 if their isn't
    pattern = re.compile('^([\d-]*)([\s]+)')
    try:
        number = pattern.search(address).group(1)
        return int(number)
    except:
        return -1

#from "this is a lit"s python version by rakhlin
def singleValueConvert(df1,df2,column):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= 1, column] = -1
    return df1, df2

In [4]:
#new feature for the street_address, use them instead of the original one
train_df["street_name"] = train_df["street_address"].apply(proecessStreet)
test_df["street_name"] = test_df["street_address"].apply(proecessStreet)

In [8]:
#new feature for the street_address, use them instead of the original one
train_df["display_street"] = train_df["display_address"].apply(proecessStreet)
test_df["display_street"] = test_df["display_address"].apply(proecessStreet)

In [9]:
train_df["street_address"] = train_df["street_address"].apply(lambda x:x.strip().lower())
test_df["street_address"] = test_df["street_address"].apply(lambda x:x.strip().lower())
train_df["display_address"] = train_df["display_address"].apply(lambda x:x.strip().lower())
test_df["display_address"] = test_df["display_address"].apply(lambda x:x.strip().lower())

In [27]:
#use this one
train_df['detail_simi'] = map(similar,train_df["display_address"],train_df["street_address"])
train_df['inSameStreet'] = train_df['detail_simi'].apply(lambda x : 1 if x > 0.5 else 0)

In [13]:
train_df['processed_simi']= map(similar,train_df["display_street"],train_df["street_name"])

In [29]:
train_df.ix[train_df['processed_simi']<0.5,['display_address','street_address','detail_simi','processed_simi','display_street','street_name']]

,display_address,street_address,detail_simi,processed_simi,display_street,street_name
100102,e 79,350 e 79,0.666667,0.444444,79,east 79
100132,w 80's & broadway,247 w 87th st.,0.250000,0.214286,80s broadway,west 87 street
100237,e 71,323 e 71,0.666667,0.444444,71,east 71
100262,midtown west,329 w 44th st,0.320000,0.307692,midtown west,west 44 street
100433,downtown brooklyn,10 city point,0.200000,0.222222,downtown brooklyn,city point
100544,,80 madison avenue,0.000000,0.000000,,madison avenue
100567,midtown west,66 west 38th street,0.322581,0.307692,midtown west,west 38 street
100640,beautiful spacious renovated modern kitchen & ...,600 w 140,0.050000,0.076923,beautiful spacious renovated modern kitchen b...,west 140
10070,e 14th st,332 e 14th st #2ab,0.666667,0.434783,14 street,east 14 st 2ab
100793,e 87,210 e 87,0.666667,0.444444,87,east 87


In [18]:
train_df.ix[(train_df['detail_simi']>0.5)&(train_df['detail_simi']<0.75),['display_address','street_address','detail_simi','processed_simi','display_street','street_name']]

,display_address,street_address,detail_simi,processed_simi,display_street,street_name
100102,e 79,350 e 79,0.666667,0.444444,79,east 79
100112,1 ave,1570 1 ave,0.666667,0.545455,ave,1 avenue
100124,3rd ave and east 22nd street,210 east 22nd st #1b,0.541667,0.540541,3 ave and east 22 street,east 22 st 1b
100136,118th st,83-43 118th st,0.727273,1.000000,118 street,118 street
100140,260th st,74-21 260th st,0.727273,1.000000,260 street,260 street
100166,4 ave,145 4 ave,0.714286,0.545455,ave,4 avenue
100221,65th ave,102-32 65th ave,0.695652,1.000000,65 avenue,65 avenue
100237,e 71,323 e 71,0.666667,0.444444,71,east 71
100282,23rd ave,210-28 23rd ave,0.695652,1.000000,23 avenue,23 avenue
100287,ave a,115 ave a,0.714286,1.000000,ave a,ave a


In [28]:
train_df['inSameStreet'] 

10        1
10000     1
100004    1
100007    1
100013    1
100014    1
100016    1
100020    1
100026    1
100027    1
100030    1
10004     1
100044    1
100048    1
10005     1
100051    1
100052    1
100053    1
100055    1
100058    1
100062    1
100063    1
100065    1
100066    1
10007     1
100071    1
100075    1
100076    1
100079    1
100081    1
         ..
99915     1
99917     1
99919     1
99921     1
99923     1
99924     1
99931     1
99933     1
99935     1
99937     1
9994      1
99953     1
99956     1
99960     1
99961     1
99964     1
99965     1
99966     1
99979     1
99980     1
99982     1
99984     1
99986     1
99987     1
99988     0
9999      1
99991     1
99992     1
99993     1
99994     1
Name: inSameStreet, dtype: int64